1. 在Scrapy目录中创建关于知乎的爬虫
    * scrapy genspider zhihu www.zhihu.com
2. 知乎在未登录的状态下不能访问其中的页面,所以在爬取知乎数据之前需要完成登录操作
    * 需要重载**scrapy.Spider**的中的**start_requests()** 函数(
        * 所有的url本质上初始时都会被start_requests()处理,只是如果不主动覆写start_requests()的话就使用默认的方法,在start_requests()里通过调用yield把处理结果送给pipline.py里继续处理
        * **start_requests()**:此方法用于生成初始请求,他**必须返回一个可迭代的对象(yield x ; return [x])**.此方法会**默认使用start_urls里面的URL来构造Request**,而且Request是GET请求方式.如果我们想在启动时以POST方式来访问某个站点,可以直接重写这个方法,**发送POST请求时使用scrapy.FormRequest**即可
            ```
        def start_requests(self):
            if method_is_overridden(cls, Spider, 'make_requests_from_url'):
                for url in self.start_urls:
                    yield self.make_requests_from_url(url)
            else:
                for url in self.start_urls:
                    yield Request(url, dont_filter=True)
            ```
            * **scrapy.FormRequest**的参数跟Request()的参数相同(headers,url,data,callback...),只是其中有个叫**formdata**的参数,实际上就是Request()中的data
        * 由上可知重写start_requests()原因有几点:
            * scrapy默认的start_urls是固定的的,现在假设有这么个需求：爬虫需要先从数据库里面读取目标URL再依次进行爬取，这时候固定的start_urls就显得不够灵活了
            * scrapy中Request默认的请求方式是GET,如果我们需要用POST请求就得重写start_requests()
            * 重写start_requests()可以满足自定义的需求(比如在Request()中传入params,headers...),这些是默认start_requests()无法做到的
    * **只有当访问完页面时才会调用callback函数**,回调函数接收访问完页面时返回的response 
    * 没有callback(self, response)函数时会默认调用**parse(self, response)**方法,这也解释了为什么如果没有重写start_requests()的话会去调用parse(),因为默认的start_requests()中的Request()没有callback函数
    * 使用scrapy时不需要使用session.cookie.save()来保存cookie值,因为scrapy自动将cookie放在Request()中并持续整个爬取流程,所以当我们请求完一个URL时直接可以请求下一个URL,只要我们一开始就获取了cookie值(保存有个人登录信息),之后跟个人信息有关的所有页面都能够访问
            
    ```
    class ZhihuSpider(scrapy.Spider):
        name = 'zhihu'
        allowed_domains = ['www.zhihu.com']
        start_urls = ['http://www.zhihu.com/']
        headers = {
            "HOST": "www.zhihu.com",
            "Referer": "https://www.zhizhu.com",
            'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0"
        }

        def start_requests(self):
            pass
    ```
    ```
    def start_requests(self):
        return [scrapy.Request('https://www.zhihu.com/#signin', headers=self.headers, callback=self.login)]
    ```
    ```
    def login(self, response):
        response_text = response.text
        match_obj = re.match('.*name="_xsrf" value="(.*?)"', response_text, re.DOTALL)
        xsrf = ''
        if match_obj:
            xsrf = (match_obj.group(1))

        if xsrf:
            post_url = "https://www.zhihu.com/login/phone_num"
            post_data = {
                "_xsrf": xsrf,
                "phone_num": "18782902568",
                "password": "admin123"
            }

            return [scrapy.FormRequest(
                url = post_url,
                formdata = post_data,
                headers=self.headers,
                callback=self.check_login
            )]
    ```
    ```
    def check_login(self, response):
        #验证服务器的返回数据判断是否成功
        text_json = json.loads(response.text)
        if "msg" in text_json and text_json["msg"] == "登录成功":
            for url in self.start_urls:
                yield scrapy.Request(url, dont_filter=True, headers=self.headers)
    ```
    ##### 所谓的URL去重，就是爬虫将重复抓取的URL去除，避免多次抓取同一网页。
    **dont_filter=True**表示可以对重复的URL地址进行爬取
    ##### 登陆成功后再次访问 http://www.zhihu.com 即可成功,因为check_login()中的Request()没有指定callback,所以在parse()中进行后续操作